In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [ ]:
# Network and training parameters.
EPOCHS = 200
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10 # number of outputs = number of digits
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2 

In [ ]:
# Loading MNIST dataset.
#  You can verify that the split between train and test is 60,000, and 10,000 respectively.
# Labels have one-hot representation.is automatically applied
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
# X_train is 60000 rows of 28x28 values; we --> reshape it to
# 60000 x 784.
RESHAPED = 784

X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# Normalize inputs to be within in [0, 1].
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
# One-hot representation of the labels.
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

11501568/11490434 [==============================] - 0s 0us/step
60000 train samples
10000 test samples


In [ ]:
def create_model(optimizer="adam"):
  model = tf.keras.models.Sequential()
  model.add(keras.layers.Dense(N_HIDDEN,
                               input_shape=(RESHAPED,),
                               name='dense_layer', activation='relu'))
  model.add(keras.layers.Dense(N_HIDDEN,
                               name='dense_layer_2', activation='relu'))
  model.add(keras.layers.Dense(NB_CLASSES,
                               name='dense_layer_3', activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
  return model


In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=create_model, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=VERBOSE)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  


In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential

In [ ]:
optimizer = ['SGD', 'adam','RMSProp']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model,param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/200
469/469 [==============================] - 3s 5ms/step - loss: 0.3139 - accuracy: 0.9104
Epoch 2/200
469/469 [==============================] - 2s 5ms/step - loss: 0.1251 - accuracy: 0.9624
Epoch 3/200
469/469 [==============================] - 2s 5ms/step - loss: 0.0861 - accuracy: 0.9735
Epoch 4/200
469/469 [==============================] - 2s 5ms/step - loss: 0.0641 - accuracy: 0.9800
Epoch 5/200
469/469 [==============================] - 2s 5ms/step - loss: 0.0525 - accuracy: 0.9832
Epoch 6/200
469/469 [==============================] - 2s 5ms/step - loss: 0.0404 - accuracy: 0.9875
Epoch 7/200
469/469 [==============================] - 2s 5ms/step - loss: 0.0328 - accuracy: 0.9898
Epoch 8/200
469/469 [==============================] - 2s 5ms/step - loss: 0.0269 - accuracy: 0.9916
Epoch 9/200
469/469 [==============================] - 2s 5ms/step - loss: 0.0217 - accuracy: 0.9930
Epoch 10/200
469/469 [==============================] - 2s 5ms/step - loss: 0.0183 - accura

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.977467 using {'optimizer': 'adam'}
0.970267 (0.000519) with: {'optimizer': 'SGD'}
0.977467 (0.000944) with: {'optimizer': 'adam'}
0.975800 (0.000884) with: {'optimizer': 'RMSProp'}
